In [1]:
import glob
import json
import numpy as np
from utils.analysis import Extrapolation

import pandas as pd
import warnings
warnings.simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [3]:
TAG = "btagsf_fix"
# TAG = "max_detajj" # DEBUG
babies = glob.glob(f"../analysis/studies/vbswh/output_{TAG}/Run2/*.root")
babies = [baby for baby in babies if "Lambda" not in baby and "VBSWH_SM" not in baby and "VVJJ" not in baby]
data_babies = [baby for baby in babies if "data.root" in baby]
sig_babies = [baby for baby in babies if "VBSWH_mkW" in baby]
bkg_babies = [baby for baby in babies if "VBSWH" not in baby and "data.root" not in baby]
print(data_babies)
print(sig_babies)
bkg_babies

['../analysis/studies/vbswh/output_btagsf_fix/Run2/data.root']
['../analysis/studies/vbswh/output_btagsf_fix/Run2/VBSWH_mkW.root']


['../analysis/studies/vbswh/output_btagsf_fix/Run2/TTbar2L.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/WJets.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/TTbar1L.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/SingleTop.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/Bosons.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/EWKWLep.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/VH.root',
 '../analysis/studies/vbswh/output_btagsf_fix/Run2/TTX.root']

In [4]:
vbswh = Extrapolation(
    sig_root_files=sig_babies,
    bkg_root_files=bkg_babies,
    data_root_files=data_babies,
    ttree_name="tree",
    weight_columns=[
        "xsec_sf", "lep_id_sf", "ewkfix_sf", 
        "elec_reco_sf", "muon_iso_sf", 
        "light_btag_sf", "bc_btag_sf", 
        "pu_sf", "prefire_sf", "trig_sf", "puid_sf", 
        "xbb_sf" # applied only because Xbb > 0.9 applied everywhere for ABCD
    ],
    reweight_column="reweights",
    plots_dir=f"/home/users/jguiang/public_html/onelep_plots/{TAG}/val"
)
vbswh.df["presel_noVBS_noBVeto"] = vbswh.df.eval(
    "hbbjet_score > 0.3"
)
vbswh.df["presel_noVBS"] = vbswh.df.eval(
    "passes_bveto and hbbjet_score > 0.3"
)
vbswh.df["presel_noDetaJJ"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and hbbjet_score > 0.3"
)
vbswh.df["presel"] = vbswh.df.eval(
    "passes_bveto and M_jj > 500 and abs(deta_jj) > 3 and hbbjet_score > 0.3"
)

ORIG_EVENT_WEIGHT = vbswh.df.event_weight.values.copy()

Loading data babies: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.05it/s]


In [5]:
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
AN_numbers = {
    "PredBkg": 0,
    "PredBkgStatErr": 0,
    "PredBkgSystErr": 0,
    "ExpSig": round(vbswh.sig_count(selection=regionD)),
    "ExpSigStatErr": round(vbswh.sig_error(selection=regionD), 1),
    "ExpSigSystErr": 0,
    "ExpBkg": round(vbswh.bkg_count(selection=regionD)),
    "BkgEstABMC": 0,
    "BkgEstABMCErr": 0,
    "BkgEstABData": 0,
    "BkgEstABDataErr": 0,
    "PredBkgMC": 0,
    "BkgEstMethodSystErr": 0,
    "BkgEstBkgCompSystErr": 0,
    "BkgEstTotalSystErr": 0,
    "BkgEstStatErr": 0,
    "BkgEstWJetsUpABMC": 0,
    "BkgEstWJetsUpABMCErr": 0,
    "BkgEstWJetsDownABMC": 0,
    "BkgEstWJetsDownABMCErr": 0,
    "BkgEstWJetsCompSyst": 0,
    "BkgEstBosonsUpABMC": 0,
    "BkgEstBosonsUpABMCErr": 0,
    "BkgEstBosonsDownABMC": 0,
    "BkgEstBosonsDownABMCErr": 0,
    "BkgEstBosonsCompSyst": 0,
    "PNetSidebandPredData": 0,
    "PNetSidebandPredDataStatErr": 0,
    "PNetSidebandData": 0,
    "PNetSidebandDataStatErr": 0,
    "SRTwoPredBkg": 0,
    "SRTwoPredBkgStatErr": 0,
    "SRTwoPredBkgSystErr": 0,
    "SRTwoBkgEstStatErr": 0,
    "SRTwoBkgEstSystErr": 0,
    "SRTwoExpSig": round(vbswh.sig_count(selection=f"{regionD} and ST > 1500")),
    "SRTwoExpSigStatErr": round(vbswh.sig_error(selection=f"{regionD} and ST > 1500"), 1),
    "SRTwoExpSigSystErr": 0,
    "LambdaWZeqNegOneExcl": 0
}

In [6]:
def ABCD(A, B, C, D, sample=None, blind=True):
    
    A_sig_count = vbswh.sig_count(selection=A)
    B_sig_count = vbswh.sig_count(selection=B)
    C_sig_count = vbswh.sig_count(selection=C)
    D_sig_count = vbswh.sig_count(selection=D)

    A_sig_error = vbswh.sig_error(selection=A)
    B_sig_error = vbswh.sig_error(selection=B)
    C_sig_error = vbswh.sig_error(selection=C)
    D_sig_error = vbswh.sig_error(selection=D)
    
    A_data_count = vbswh.data_count(selection=A)
    B_data_count = vbswh.data_count(selection=B)
    C_data_count = vbswh.data_count(selection=C)
    D_data_count = vbswh.data_count(selection=D)
        
    if sample is None:
        A_bkg_count = vbswh.bkg_count(selection=A)
        B_bkg_count = vbswh.bkg_count(selection=B)
        C_bkg_count = vbswh.bkg_count(selection=C)
        D_bkg_count = vbswh.bkg_count(selection=D)
        
        A_bkg_error = vbswh.bkg_error(selection=A)
        B_bkg_error = vbswh.bkg_error(selection=B)
        C_bkg_error = vbswh.bkg_error(selection=C)
        D_bkg_error = vbswh.bkg_error(selection=D)
    else:
        A_bkg_count = vbswh.sample_count(sample, selection=A)
        B_bkg_count = vbswh.sample_count(sample, selection=B)
        C_bkg_count = vbswh.sample_count(sample, selection=C)
        D_bkg_count = vbswh.sample_count(sample, selection=D)
        
        A_bkg_error = vbswh.sample_error(sample, selection=A)
        B_bkg_error = vbswh.sample_error(sample, selection=B)
        C_bkg_error = vbswh.sample_error(sample, selection=C)
        D_bkg_error = vbswh.sample_error(sample, selection=D)
        
        A_data_count -= (vbswh.bkg_count(selection=A) - A_bkg_count)
        B_data_count -= (vbswh.bkg_count(selection=B) - B_bkg_count)
        C_data_count -= (vbswh.bkg_count(selection=C) - C_bkg_count)
        D_data_count -= (vbswh.bkg_count(selection=D) - D_bkg_count)
        
    A_data_error = np.sqrt(A_data_count)
    B_data_error = np.sqrt(B_data_count)
    C_data_error = np.sqrt(C_data_count)
    D_data_error = np.sqrt(D_data_count)
    
    if blind:
        D_data_count = 0
        D_data_error = 0
        
    print("region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err")
    print(f"A,{A_bkg_count:.2f},{A_bkg_error:.2f},{A_sig_count:.2f},{A_sig_error:.2f},{A_data_count:.2f},{A_data_error:.2f}")
    print(f"B,{B_bkg_count:.2f},{B_bkg_error:.2f},{B_sig_count:.2f},{B_sig_error:.2f},{B_data_count:.2f},{B_data_error:.2f}")
    print(f"C,{C_bkg_count:.2f},{C_bkg_error:.2f},{C_sig_count:.2f},{C_sig_error:.2f},{C_data_count:.2f},{C_data_error:.2f}")
    print(f"D,{D_bkg_count:.2f},{D_bkg_error:.2f},{D_sig_count:.2f},{D_sig_error:.2f},{D_data_count:.2f},{D_data_error:.2f}")
        
    # Transfer factor
    mc_tf = A_bkg_count/B_bkg_count
    mc_tf_err = mc_tf*np.sqrt((A_bkg_error/A_bkg_count)**2 + (B_bkg_error/B_bkg_count)**2)
    
    data_tf = A_data_count/B_data_count
    data_tf_err = data_tf*np.sqrt((A_data_error/A_data_count)**2 + (B_data_error/B_data_count)**2)
    
    # Prediction
    D_bkg_pred = mc_tf*C_bkg_count
    syst = abs(1 - D_bkg_pred/D_bkg_count)
    print(syst)
    
    # Transfer factor correction
    mc_tf_fix = (D_bkg_count/C_bkg_count)/mc_tf
    print(f"Calculated MC TF fix: {mc_tf_fix}")
    
    mc_tf *= mc_tf_fix
    data_tf *= mc_tf_fix
    
    # Prediction
    D_bkg_pred = mc_tf*C_bkg_count
    D_bkg_pred_err = D_bkg_pred*np.sqrt(
        (A_bkg_error/A_bkg_count)**2 + (B_bkg_error/B_bkg_count)**2 + (C_bkg_error/C_bkg_count)**2
    )
    
    D_data_pred = data_tf*C_data_count
    D_data_pred_err = D_data_pred*np.sqrt(
        (A_data_error/A_data_count)**2 + (B_data_error/B_data_count)**2 + (C_data_error/C_data_count)**2
    )
    
    print(f"D_pred,{D_bkg_pred:.2f},{D_bkg_pred_err:.2f},,,{D_data_pred:.2f},{D_data_pred_err:.2f}")
    
    return D_data_pred, D_data_pred_err, syst*D_data_pred

In [7]:
# Regular ABCD
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=None)
print()

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,173.09,3.25,6.93,0.40,142.00,11.92
B,242.52,5.87,0.27,0.08,201.00,14.18
C,181.77,4.42,11.63,0.52,170.00,13.04
D,116.61,3.84,366.39,2.92,0.00,0.00
0.11251152716341606
Calculated MC TF fix: 0.8988670908873293
D_pred,116.61,4.56,,,107.95,14.44



In [8]:
# Regular ABCD
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

vbswh.df["event_weight"] = ORIG_EVENT_WEIGHT*vbswh.df.ps_isr2p0_fsr1p0
ABCD(regionA,regionB,regionC,regionD,sample=None)

print()

vbswh.df["event_weight"] = ORIG_EVENT_WEIGHT*vbswh.df.ps_isr0p5_fsr1p0
ABCD(regionA,regionB,regionC,regionD,sample=None)

vbswh.df["event_weight"] = ORIG_EVENT_WEIGHT

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,163.13,3.07,6.91,0.40,142.00,11.92
B,238.94,5.92,0.29,0.09,201.00,14.18
C,179.29,4.30,11.78,0.53,170.00,13.04
D,110.80,3.67,367.71,2.94,0.00,0.00
0.10474974925100633
Calculated MC TF fix: 0.9051823733637195
D_pred,110.80,4.35,,,108.71,14.54

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,185.64,3.55,6.95,0.40,142.00,11.92
B,246.45,5.83,0.26,0.08,201.00,14.18
C,184.61,4.65,11.45,0.52,170.00,13.04
D,123.69,4.14,364.35,2.91,0.00,0.00
0.12423239709125622
Calculated MC TF fix: 0.889495804059121
D_pred,123.69,4.89,,,106.83,14.29


In [9]:
sample = "VVJJ"

# Regular ABCD
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,0.00,0.00,6.93,0.40,-31.09,nan
B,0.00,0.00,0.27,0.08,-41.52,nan
C,0.00,0.00,11.63,0.52,-11.77,nan
D,0.00,0.00,366.39,2.92,0.00,0.00
nan
Calculated MC TF fix: nan
D_pred,nan,nan,,,nan,nan


/tmp/ipykernel_1722739/1050822872.py:44: RuntimeWarning: invalid value encountered in sqrt
  A_data_error = np.sqrt(A_data_count)
/tmp/ipykernel_1722739/1050822872.py:45: RuntimeWarning: invalid value encountered in sqrt
  B_data_error = np.sqrt(B_data_count)
/tmp/ipykernel_1722739/1050822872.py:46: RuntimeWarning: invalid value encountered in sqrt
  C_data_error = np.sqrt(C_data_count)
/tmp/ipykernel_1722739/1050822872.py:60: RuntimeWarning: invalid value encountered in double_scalars
  mc_tf = A_bkg_count/B_bkg_count
/tmp/ipykernel_1722739/1050822872.py:61: RuntimeWarning: invalid value encountered in double_scalars
  mc_tf_err = mc_tf*np.sqrt((A_bkg_error/A_bkg_count)**2 + (B_bkg_error/B_bkg_count)**2)
/tmp/ipykernel_1722739/1050822872.py:72: RuntimeWarning: invalid value encountered in double_scalars
  mc_tf_fix = (D_bkg_count/C_bkg_count)/mc_tf
/tmp/ipykernel_1722739/1050822872.py:81: RuntimeWarning: invalid value encountered in double_scalars
  (A_bkg_error/A_bkg_count)**2 + (B_b

(nan, nan, nan)

In [10]:
# Doing ABCD only in MSD > 80

sample = None

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop > 80"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,173.09,3.25,6.93,0.40,142.00,11.92
B,242.52,5.87,0.27,0.08,201.00,14.18
C,136.66,4.13,11.19,0.51,122.00,11.05
D,88.57,3.38,352.36,2.86,0.00,0.00
0.1011892235904901
Calculated MC TF fix: 0.9081091410788085
D_pred,88.57,3.81,,,78.27,11.13



In [11]:
# Doing ABCD only in detajj sideband

sample = None

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs_deta_jj < 4"
regionA = f"{SRlike} and abs(deta_jj) > 2 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 2 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 2 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 2 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample,blind=False)
print()

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,177.53,5.53,0.27,0.08,154.00,12.41
B,64.94,1.95,0.00,0.00,47.00,6.86
C,49.96,2.14,0.19,0.07,34.00,5.83
D,131.78,3.87,11.44,0.52,136.00,11.66
0.036314552103454245
Calculated MC TF fix: 0.9649579830469956
D_pred,131.78,8.04,,,107.50,25.71



In [12]:
# Doing ABCD only in 50 < MSD < 150

sample = None

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop < 150"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 100"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 100"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 100"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 100"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,52.18,3.02,40.84,0.97,56.00,7.48
B,90.83,3.48,1.27,0.17,82.00,9.06
C,90.94,2.73,10.37,0.49,88.00,9.38
D,64.43,2.36,325.55,2.75,0.00,0.00
0.18911811632701625
Calculated MC TF fix: 1.233225233088676
D_pred,64.43,4.88,,,74.11,15.08



In [13]:
sample = None

# Regular ABCD
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()

# Doing ABCD for MSD > 150
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 200"  # A1
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 200" # B1
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"  # B2
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"   # A2

ABCD(regionA,regionB,regionC,regionD,sample=sample,blind=False)
print()

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop < 250"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 200"  # A1
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 200" # B1
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"  # B2
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"   # A2

ABCD(regionA,regionB,regionC,regionD,sample=sample,blind=False)
print()

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop < 250"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 180"  # A1
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 180" # B1
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 180"  # B2
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 180"   # A2

ABCD(regionA,regionB,regionC,regionD,sample=sample,blind=False)
print()

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,173.09,3.25,6.93,0.40,142.00,11.92
B,242.52,5.87,0.27,0.08,201.00,14.18
C,181.77,4.42,11.63,0.52,170.00,13.04
D,116.61,3.84,366.39,2.92,0.00,0.00
0.11251152716341606
Calculated MC TF fix: 0.8988670908873293
D_pred,116.61,4.56,,,107.95,14.44

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,41.85,1.72,0.29,0.08,28.00,5.29
B,96.30,4.46,0.03,0.02,70.00,8.37
C,146.22,3.81,0.24,0.08,131.00,11.45
D,131.24,2.76,6.64,0.39,114.00,10.68
0.5158628363138185
Calculated MC TF fix: 2.065530339348626
D_pred,131.24,8.82,,,108.23,25.98

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,26.72,1.23,0.18,0.06,22.00,4.69
B,54.79,4.22,0.02,0.01,40.00,6.32
C,146.22,3.81,0.24,0.08,131.00,11.45
D,131.24,2.76,6.64,0.39,114.00,10.68
0.456705130407402
Calculated MC TF fix: 1.840621099090946
D_pred,131.24,12.26,,,132.62,37.06

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,86.22,2.20,0.2

In [14]:
sample = "TTbar1L"

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop < 200"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()

SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (hbbjet_msoftdrop > 200 or hbbjet_msoftdrop < 150)"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

ABCD(regionA,regionB,regionC,regionD,sample=sample)
print()


SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 200"  # A1
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 200" # B1
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"  # B2
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"   # A2

ABCD(regionA,regionB,regionC,regionD,sample=sample,blind=False)

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,136.74,2.63,6.93,0.40,105.65,10.28
B,169.11,2.84,0.27,0.08,127.58,11.30
C,59.39,1.69,11.63,0.52,47.62,6.90
D,49.95,1.58,366.39,2.92,0.00,0.00
0.03846793363152079
Calculated MC TF fix: 1.0400069170618582
D_pred,49.95,1.91,,,41.01,8.03

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,109.73,2.36,6.64,0.39,92.49,9.62
B,109.48,2.31,0.24,0.08,94.26,9.71
C,59.39,1.69,11.63,0.52,47.62,6.90
D,49.95,1.58,366.39,2.92,0.00,0.00
0.19185248778038555
Calculated MC TF fix: 0.8390300060222412
D_pred,49.95,2.07,,,39.20,8.07

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,26.92,1.15,0.29,0.08,13.18,3.63
B,59.41,1.65,0.03,0.02,33.39,5.78
C,59.39,1.69,11.63,0.52,47.62,6.90
D,49.95,1.58,366.39,2.92,0.00,0.00
0.4612460615584769
Calculated MC TF fix: 1.8561349229162825
D_pred,49.95,2.91,,,34.88,12.42

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,27.01,1.15,0.29,0.08,13.17,3.

In [15]:
pred, stat, syst = vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
#     SRlike,
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and ((is_data or is_signal) or (name == 'TTbar1L'))",
#     "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (hbbjet_msoftdrop > 200 or hbbjet_msoftdrop < 150)",
    h_dir="left", v_dir="up", 
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and ((is_data or is_signal) or (name == 'TTbar1L')) and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,136.7437246819876,2.625319769465052,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and ((is_data or is_signal) or (name == 'TTbar1L')) and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,169.1066296395771,2.840797400309562,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and ((is_data or is_signal) or (name == 'TTbar1L')) and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,59.389843911136694,1.688793572473669,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and ((is_data or is_signal) or (name == 'TTb

# Calculations for review

In [16]:
def get_closure_data(regionA, regionB, regionC, regionD):
    # Counts
    A_count = vbswh.data_count(selection=regionA)
    B_count = vbswh.data_count(selection=regionB)
    C_count = vbswh.data_count(selection=regionC)
    D_count = vbswh.data_count(selection=regionD)
    
    # Errors
    A_error = vbswh.data_error(selection=regionA)
    B_error = vbswh.data_error(selection=regionB)
    C_error = vbswh.data_error(selection=regionC)
    D_error = vbswh.data_error(selection=regionD)
        
    # Transfer factor
    tf = A_count/B_count
    tf_err = tf*np.sqrt((A_error/A_count)**2 + (B_error/B_count)**2)
    
    # Prediction
    D_pred = tf*C_count
    D_pred_err = D_pred*np.sqrt((A_error/A_count)**2 + (B_error/B_count)**2 + (C_error/C_count)**2)
    
    print("For cross check with vbswh.ABCD:")
    print("A", A_count)
    print("B", B_count)
    print("C", C_count)
    
    return D_count, D_error, D_pred, D_pred_err

In [17]:
pred, stat, syst = vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 200",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop >= 150",
    h_dir="left", v_dir="up", 
    show_data=True
)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop >= 150 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 200)),A,41.84830433152469,1.7220754875636417,0.2890407211789309,0.08429115700561285,28,5.291502622129181
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop >= 150 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 200)),B,96.30371805340148,4.457218679733056,0.0288969219566418,0.01684186052315507,70,8.366600265340756
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop >= 150 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 200,C,146.21862897934523,3.8145583172479225,0.24430361449008076,0.08277073980790753,131,11.445523142259598
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and hbbjet_msoftdrop >= 150 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 200,D,131.24086879783061,2.761552921006866,

In [18]:
# Test closure in data in AB
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 200"  # A1
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 200" # B1
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"  # B2
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150 and hbbjet_msoftdrop < 200"   # A2
print(get_closure_data(regionA, regionB, regionC, regionD))

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

For cross check with vbswh.ABCD:
A 28
B 70
C 131
(114, 10.677078252031311, 52.400000000000006, 12.57966613229461)


In [19]:
vbswh.df.event_weight /= vbswh.df.xbb_sf

pred, stat, syst = vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3",
    h_dir="left", v_dir="up", 
    show_data=True
)

vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,80.20758081946934,2.6839698255445334,0.08439702220199205,0.03950353213123246,87,9.327379053088816
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,120.09455151579851,2.665647656470602,0.038835960043278994,0.038835960043278994,119,10.908712114635714
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,164.64765034353124,7.282772225355114,0.0619171498826447,0.036425593838826625,188,13.711309200802088
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,96.94197173080359,4.052922190009257,5.464193655

In [20]:
table = """
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,80.13847261270763,2.6816135654763578,0.08432259475889595,0.039495855428519695,87,9.327379053088816
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,119.61053554308242,2.6484914913282003,0.03545809722221798,0.03545809722221798,119,10.908712114635714
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,164.07787245080965,7.259507711411256,0.06196954418901307,0.03641677545529165,188,13.711309200802088
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,96.88083927555962,4.048700597049295,5.463165849701192,0.3450759994857886,BLINDED,BLINDED
"""

print("Region & Total bkg. (MC) &    Total sig.   &   Total data    \\\\")
print("\\hline")
for line in table.splitlines()[2:]:
    cut, region, bkg_wgt, bkg_err, sig_wgt, sig_err, data, data_err = line.split(",")
    bkg_wgt, bkg_err = (float(bkg_wgt), float(bkg_err))
    sig_wgt, sig_err = (float(sig_wgt), float(sig_err))
    if region == "D":
        print(f"{region:^6} & ${bkg_wgt:>5.1f} \pm {bkg_err:<4.1f}$ & ${sig_wgt:>4.1f} \pm {sig_err:<4.1f}$ & {'--':^14} \\\\")
    else:
        data, data_err = (int(data), float(data_err))
        print(f"{region:^6} & ${bkg_wgt:>5.1f} \pm {bkg_err:<4.1f}$ & ${sig_wgt:>4.1f} \pm {sig_err:<4.1f}$ & ${data:>3} \pm {data_err:<4.1f}$ \\\\")

Region & Total bkg. (MC) &    Total sig.   &   Total data    \\
\hline
  A    & $ 80.1 \pm 2.7 $ & $ 0.1 \pm 0.0 $ & $ 87 \pm 9.3 $ \\
  B    & $119.6 \pm 2.6 $ & $ 0.0 \pm 0.0 $ & $119 \pm 10.9$ \\
  C    & $164.1 \pm 7.3 $ & $ 0.1 \pm 0.0 $ & $188 \pm 13.7$ \\
  D    & $ 96.9 \pm 4.0 $ & $ 5.5 \pm 0.3 $ &       --       \\


In [21]:
# Test closure in data in PNet sideband
# SRlike = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.9 and hbbjet_score > 0.7"
SRlike = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score <= 0.5 and hbbjet_score > 0.3"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"
D, D_err, D_pred, D_pred_err = get_closure_data(regionA, regionB, regionC, regionD)
print(D, D_err, D_pred, D_pred_err)

AN_numbers["PNetSidebandPredData"] = round(D_pred, 1)
AN_numbers["PNetSidebandPredDataStatErr"] = round(D_pred_err, 1)
AN_numbers["PNetSidebandData"] = D
AN_numbers["PNetSidebandDataStatErr"] = round(D_err, 1)

# Reset
SRlike  = "presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9"
regionA = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop >= 150"
regionB = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop >= 150"
regionC = f"{SRlike} and abs(deta_jj) <= 4 and hbbjet_msoftdrop < 150"
regionD = f"{SRlike} and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150"

For cross check with vbswh.ABCD:
A 87
B 119
C 188
118 10.862780491200215 137.44537815126048 21.8260331349876


In [22]:
def get_closure_numbers(regionA, regionB, regionC, region_D, sample=None):
    if sample is None:
        A_count = vbswh.bkg_count(selection=regionA)
        B_count = vbswh.bkg_count(selection=regionB)
        C_count = vbswh.bkg_count(selection=regionC)
        D_count = vbswh.bkg_count(selection=regionD)
        
        A_error = vbswh.bkg_error(selection=regionA)
        B_error = vbswh.bkg_error(selection=regionB)
        C_error = vbswh.bkg_error(selection=regionC)
        D_error = vbswh.bkg_error(selection=regionD)
    else:
        A_count = vbswh.sample_count(sample, selection=regionA)
        B_count = vbswh.sample_count(sample, selection=regionB)
        C_count = vbswh.sample_count(sample, selection=regionC)
        D_count = vbswh.sample_count(sample, selection=regionD)
        
        A_error = vbswh.sample_error(sample, selection=regionA)
        B_error = vbswh.sample_error(sample, selection=regionB)
        C_error = vbswh.sample_error(sample, selection=regionC)
        D_error = vbswh.sample_error(sample, selection=regionD)
        
    # Transfer factor
    tf = A_count/B_count
    tf_err = tf*np.sqrt((A_error/A_count)**2 + (B_error/B_count)**2)
    
    # Prediction
    D_pred = tf*C_count
    D_pred_err = D_pred*np.sqrt((A_error/A_count)**2 + (B_error/B_count)**2 + (C_error/C_count)**2)
    
    # Systematic
    syst = abs(1 - D_pred/D_count)
    
    return tf, tf_err, D_pred, D_pred_err, D_count, D_error, syst

sample_tex = {
    "TTbar1L": "$t\\bar{t}+1\\ell$",
    "TTbar2L": "$t\\bar{t}+2\\ell$",
    "WJets": "W+jets",
    "SingleTop": "Single top"
}

print("Sample           & Transfer factor & $D_{MC}^{pred}$   & $D_{MC}$          & $|1 - D_{MC}^{pred}/D_{MC}|$")
print("\\hline")
for sample in [None, "TTbar1L", "TTbar2L", "WJets", "SingleTop", "Bosons"]:
    tf, tf_err, D_pred, D_pred_err, D, D_err, syst = get_closure_numbers(
        regionA, regionB, regionC, regionD, sample=sample
    )
    if sample is None:
        sample = "Total bkg."
    print(
        f"{sample_tex.get(sample, sample):<16}"
        f" & ${tf:.2f} \pm {tf_err:.2f}$"
        f" & ${D_pred:>6.1f} \pm {D_pred_err:<4.1f}$"
        f" & ${D:>6.1f} \pm {D_err:<4.1f}$"
        f" & ${syst*100:>5.1f}$"
    )

Sample           & Transfer factor & $D_{MC}^{pred}$   & $D_{MC}$          & $|1 - D_{MC}^{pred}/D_{MC}|$
\hline
Total bkg.       & $0.71 \pm 0.02$ & $ 129.7 \pm 5.1 $ & $ 116.6 \pm 3.8 $ & $ 11.3$
$t\bar{t}+1\ell$ & $0.81 \pm 0.02$ & $  48.0 \pm 1.8 $ & $  49.9 \pm 1.6 $ & $  3.8$
$t\bar{t}+2\ell$ & $0.33 \pm 0.03$ & $  12.7 \pm 1.3 $ & $  21.5 \pm 0.9 $ & $ 40.8$
W+jets           & $0.37 \pm 0.03$ & $  15.8 \pm 1.1 $ & $  21.2 \pm 1.3 $ & $ 25.5$
Single top       & $1.16 \pm 0.16$ & $  17.2 \pm 3.0 $ & $  13.1 \pm 1.4 $ & $ 30.8$
Bosons           & $0.13 \pm 0.11$ & $   2.7 \pm 2.5 $ & $   6.0 \pm 2.6 $ & $ 54.9$


# Main ABCD

In [23]:
D_pred, D_pred_stat_err, D_pred_syst_err = ABCD(regionA, regionB, regionC, regionD)

print(D_pred, D_pred_stat_err, D_pred_syst_err)

AN_numbers["PredBkg"] = round(D_pred)
AN_numbers["PredBkgStatErr"] = round(D_pred_stat_err, 1)

region,bkg_count,bkg_error,sig_count,sig_error,data_count,data_err
A,173.09,3.25,6.93,0.40,142.00,11.92
B,242.52,5.87,0.27,0.08,201.00,14.18
C,181.77,4.42,11.63,0.52,170.00,13.04
D,116.61,3.84,366.39,2.92,0.00,0.00
0.11251152716341606
Calculated MC TF fix: 0.8988670908873293
D_pred,116.61,4.56,,,107.95,14.44
107.95349041801059 14.443084064645305 12.146012069551574


In [24]:
pred, stat, syst = vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)
AN_numbers["PredBkgNoCorrection"] = round(pred)
AN_numbers["PredBkgNoCorrectionStatErr"] = round(pred*stat, 1)
A_bkg_wgt = vbswh.bkg_count(selection=regionA)
A_bkg_err = vbswh.bkg_error(selection=regionA)
B_bkg_wgt = vbswh.bkg_count(selection=regionB)
B_bkg_err = vbswh.bkg_error(selection=regionB)
AN_numbers["PredBkgMC"] = round(A_bkg_wgt/B_bkg_wgt*vbswh.bkg_count(selection=regionC), 1)
AN_numbers["BkgEstABMC"] = A_bkg_wgt/B_bkg_wgt
AN_numbers["BkgEstABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
AN_numbers["BkgEstMethodSystErr"] = syst*100
AN_numbers["BkgEstStatErr"] = stat*100
A_data     = vbswh.data_count(selection=regionA)
A_data_err = vbswh.data_error(selection=regionA)
B_data     = vbswh.data_count(selection=regionB)
B_data_err = vbswh.data_error(selection=regionB)
AN_numbers["BkgEstABData"] = A_data/B_data
AN_numbers["BkgEstABDataErr"] = round(np.sqrt((B_data_err/B_data)**2 + (A_data_err/A_data)**2)*100, 1)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,173.0891731293553,3.2544920525927097,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,242.5223470327467,5.866656076049336,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,181.77245485083276,4.424430070302483,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,116.6115837178872,3.8394256994178146,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_MC,0.7137040163395081,0.0306381211

In [25]:
table = """cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,173.0891731293553,3.2544920525927097,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,242.52234703274672,5.8666560760493365,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,181.77245485083276,4.424430070302483,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,116.61158371788719,3.8394256994178146,366.39291471328704,2.9166173749469424,BLINDED,BLINDED"""

print("Region & Total bkg. (MC) &    Total sig.   &   Total data   \\\\")
print("\\hline")
for line in table.splitlines()[1:]:
    cut, region, bkg_wgt, bkg_err, sig_wgt, sig_err, data, data_err = line.split(",")
    bkg_wgt, bkg_err = (float(bkg_wgt), float(bkg_err))
    sig_wgt, sig_err = (float(sig_wgt), float(sig_err))
    if region == "D":
        print(f"{region:^6} & ${bkg_wgt:.1f} \pm {bkg_err:.1f}$ & ${sig_wgt:.1f} \pm {sig_err:.1f}$ & {'--':^14} \\\\")
    else:
        data, data_err = (int(data), float(data_err))
        print(f"{region:^6} & ${bkg_wgt:.1f} \pm {bkg_err:.1f}$ & ${sig_wgt:>5.1f} \pm {sig_err:.1f}$ & ${data} \pm {data_err:.1f}$ \\\\")

Region & Total bkg. (MC) &    Total sig.   &   Total data   \\
\hline
  A    & $173.1 \pm 3.3$ & $  6.9 \pm 0.4$ & $142 \pm 11.9$ \\
  B    & $242.5 \pm 5.9$ & $  0.3 \pm 0.1$ & $201 \pm 14.2$ \\
  C    & $181.8 \pm 4.4$ & $ 11.6 \pm 0.5$ & $170 \pm 13.0$ \\
  D    & $116.6 \pm 3.8$ & $366.4 \pm 2.9$ &       --       \\


# W+jets systematic

In [27]:
for factor in [1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:

    print(f"------- up/down by {factor} -------")

    vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= factor
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )

    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    print("")

    vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= (1/factor)
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )

    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    
    print(f"------------------------------")

------- up/down by 1.1 -------
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,174.2198408899568,3.2712944413487284,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,245.59695529826803,5.877252282859699,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,186.07676246761534,4.447552866757465,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,118.73563775075165,3.8827329723896304,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_M

------------------------------
------- up/down by 1.7 -------
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,181.00384745356604,3.402727984261884,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,264.0446048913958,5.961343929873796,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,211.90260816831082,4.62837843606681,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,131.47996194793828,4.213195984420227,366.39291471328704,2.9166173749469424,BLINDED,BLIND

In [62]:
vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 2
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)
A_bkg_wgt = vbswh.bkg_count(selection=regionA)
A_bkg_err = vbswh.bkg_error(selection=regionA)
B_bkg_wgt = vbswh.bkg_count(selection=regionB)
B_bkg_err = vbswh.bkg_error(selection=regionB)
AN_numbers["BkgEstWJetsUpABMC"] = A_bkg_wgt/B_bkg_wgt
AN_numbers["BkgEstWJetsUpABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)

vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
print("")

vbswh.df.loc[vbswh.df.name == "WJets", "event_weight"] *= 0.5
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)
A_bkg_wgt = vbswh.bkg_count(selection=regionA)
A_bkg_err = vbswh.bkg_error(selection=regionA)
B_bkg_wgt = vbswh.bkg_count(selection=regionB)
B_bkg_err = vbswh.bkg_error(selection=regionB)
AN_numbers["BkgEstWJetsDownABMC"] = A_bkg_wgt/B_bkg_wgt
AN_numbers["BkgEstWJetsDownABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
AN_numbers["BkgEstWJetsCompSyst"] = 100*max(
    abs(1 - AN_numbers["BkgEstWJetsUpABMC"]/AN_numbers["BkgEstABMC"]),
    abs(1 - AN_numbers["BkgEstWJetsDownABMC"]/AN_numbers["BkgEstABMC"])
)

vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,184.39585073537066,3.486851010863566,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,273.2684296879597,6.016259668311026,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,224.8155310186586,4.744072627870174,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,137.8521240465316,4.417997142575125,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_MC,0.6747791940178708,0.029021959916

# Bosons systematic

In [28]:
for factor in [1.1, 1.2, 1.3, 1.5, 1.7, 2.0]:

    print(f"------- up/down by {factor} -------")

    vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= factor
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )

    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    print("")

    vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= (1/factor)
    vbswh.ABCD( 
        "abs(deta_jj) > 4",
        "hbbjet_msoftdrop < 150",
        SRlike,
        h_dir="left", v_dir="up", 
        show_data=True
    )

    vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
    
    print(f"------------------------------")

------- up/down by 1.1 -------
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,173.18977983008563,3.269421415696238,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,243.32204166251435,6.254286575100409,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,183.9256933226324,4.671107832347749,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,117.21165758464207,4.025435653202025,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_MC,

------------------------------
------- up/down by 1.7 -------
cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,173.7934200344677,3.3864877642781845,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,248.1202094411202,8.7580635390569,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,196.84512415343045,6.3062337299540125,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,120.81210078517132,5.282771258707903,366.39291471328704,2.9166173749469424,BLINDED,BLIND

In [63]:
vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= 2
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)
A_bkg_wgt = vbswh.bkg_count(selection=regionA)
A_bkg_err = vbswh.bkg_error(selection=regionA)
B_bkg_wgt = vbswh.bkg_count(selection=regionB)
B_bkg_err = vbswh.bkg_error(selection=regionB)
AN_numbers["BkgEstBosonsUpABMC"] = A_bkg_wgt/B_bkg_wgt
AN_numbers["BkgEstBosonsUpABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)

vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()
print("")

vbswh.df.loc[vbswh.df.name == "Bosons", "event_weight"] *= 0.5
vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)
A_bkg_wgt = vbswh.bkg_count(selection=regionA)
A_bkg_err = vbswh.bkg_error(selection=regionA)
B_bkg_wgt = vbswh.bkg_count(selection=regionB)
B_bkg_err = vbswh.bkg_error(selection=regionB)
AN_numbers["BkgEstBosonsDownABMC"] = A_bkg_wgt/B_bkg_wgt
AN_numbers["BkgEstBosonsDownABMCErr"] = round(np.sqrt((B_bkg_err/B_bkg_wgt)**2 + (A_bkg_err/A_bkg_wgt)**2)*100, 1)
AN_numbers["BkgEstBosonsCompSyst"] = 100*max(
    abs(1 - AN_numbers["BkgEstBosonsUpABMC"]/AN_numbers["BkgEstABMC"]),
    abs(1 - AN_numbers["BkgEstBosonsDownABMC"]/AN_numbers["BkgEstABMC"])
)

vbswh.df.event_weight = ORIG_EVENT_WEIGHT.copy()

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,174.0952401366587,3.4616638638734747,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,250.51929333042318,10.076622729905836,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,203.30483956882946,7.1852603449337575,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,122.61232238543596,5.9699554626413995,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_MC,0.6949374550048537,0.044869

# AN numbers

In [64]:
AN_numbers["BkgEstBosonsCompSyst"]

2.6294599588923995

In [65]:
AN_numbers["BkgEstBkgCompSystErr"] = np.sqrt(
    AN_numbers["BkgEstWJetsCompSyst"]**2 + AN_numbers["BkgEstBosonsCompSyst"]**2
)
AN_numbers["BkgEstTotalSystErr"] = np.sqrt(
    AN_numbers["BkgEstMethodSystErr"]**2 + AN_numbers["BkgEstBkgCompSystErr"]**2
)

In [66]:
AN_numbers["BkgEstBkgCompSystErr"]

6.054689646776229

In [67]:
# SR2 numbers for posterity
AN_numbers["SRTwoPredBkg"] = (
    AN_numbers["PredBkg"]*vbswh.bkg_count(selection=f"{regionD} and ST > 1500")/AN_numbers["PredBkgMC"]
)
AN_numbers["SRTwoBkgEstSystErr"] = np.sqrt(
    (vbswh.data_error(selection=f"{regionB} and ST > 1500")/vbswh.data_count(selection=f"{regionB} and ST > 1500"))**2
    + (AN_numbers["BkgEstTotalSystErr"]/100)**2
)
AN_numbers["SRTwoPredBkgSystErr"] = round(AN_numbers["SRTwoBkgEstSystErr"]*AN_numbers["SRTwoPredBkg"], 1)
AN_numbers["SRTwoBkgEstSystErr"] = round(AN_numbers["SRTwoBkgEstSystErr"]*100, 1)

AN_numbers["SRTwoBkgEstStatErr"] = round(AN_numbers["BkgEstStatErr"], 1)
AN_numbers["SRTwoPredBkgStatErr"] = round(AN_numbers["SRTwoPredBkg"]*(AN_numbers["BkgEstStatErr"]/100), 1)
AN_numbers["SRTwoPredBkg"] = round(AN_numbers["SRTwoPredBkg"])

AN_numbers["SRTwoExpSig"] = round(vbswh.sig_count(selection=f"{regionD} and ST > 1500"))
AN_numbers["SRTwoExpSigStatErr"] = round(vbswh.sig_error(selection=f"{regionD} and ST > 1500"), 1)

In [68]:
pred, stat, syst = vbswh.ABCD( 
    "abs(deta_jj) > 4",
    "hbbjet_msoftdrop < 150",
    SRlike,
    h_dir="left", v_dir="up", 
    show_data=True
)

AN_numbers["PredBkgNoCorrectionSystErr"] = round(pred*AN_numbers["BkgEstTotalSystErr"]/100, 1)

D_pred, D_pred_stat_err, D_pred_syst_err = ABCD(regionA, regionB, regionC, regionD)
AN_numbers["PredBkgSystErr"] = round(D_pred*AN_numbers["BkgEstTotalSystErr"]/100, 1)

cut,region,bkg_wgt,bkg_err,sig_wgt,sig_err,data,data_err
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and (not (hbbjet_msoftdrop < 150)),A,173.0891731293553,3.2544920525927097,6.925475454330245,0.3976089035112662,142,11.916375287812984
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and (not (hbbjet_msoftdrop < 150)),B,242.52234703274672,5.8666560760493365,0.27320053644672254,0.08446681972366271,201,14.177446878757825
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and (not (abs(deta_jj) > 4)) and hbbjet_msoftdrop < 150,C,181.77245485083276,4.424430070302483,11.632430039225358,0.5222217969255568,170,13.038404810405298
presel_noDetaJJ and M_jj > 600 and ST > 900 and hbbjet_score > 0.9 and abs(deta_jj) > 4 and hbbjet_msoftdrop < 150,D,116.6115837178872,3.8394256994178146,366.39291471328704,2.9166173749469424,BLINDED,BLINDED

name,extp,rel_err
BtoA_MC,0.713704016339508,0.030638121

In [69]:
AN_numbers["BkgEstABMC"] = round(AN_numbers["BkgEstABMC"], 2)
AN_numbers["BkgEstABData"] = round(AN_numbers["BkgEstABData"], 2)

AN_numbers["BkgEstWJetsUpABMC"] = round(AN_numbers["BkgEstWJetsUpABMC"], 2)
AN_numbers["BkgEstWJetsDownABMC"] = round(AN_numbers["BkgEstWJetsDownABMC"], 2)
AN_numbers["BkgEstWJetsCompSyst"] = round(AN_numbers["BkgEstWJetsCompSyst"], 1)

AN_numbers["BkgEstBosonsUpABMC"] = round(AN_numbers["BkgEstBosonsUpABMC"], 2)
AN_numbers["BkgEstBosonsDownABMC"] = round(AN_numbers["BkgEstBosonsDownABMC"], 2)
AN_numbers["BkgEstBosonsCompSyst"] = round(AN_numbers["BkgEstBosonsCompSyst"], 1)

AN_numbers["BkgEstBkgCompSystErr"] = round(AN_numbers["BkgEstBkgCompSystErr"], 1)
AN_numbers["BkgEstMethodSystErr"] = round(AN_numbers["BkgEstMethodSystErr"], 1)
AN_numbers["BkgEstTotalSystErr"] = round(AN_numbers["BkgEstTotalSystErr"], 1)
AN_numbers["BkgEstStatErr"] = round(AN_numbers["BkgEstStatErr"], 1)

In [28]:
with open("AN_numbers.json", "w") as f_out:
    json.dump(AN_numbers, f_out)

AN_numbers # must run vbswh-sys.ipynb to fill completely

{'PredBkg': 108,
 'PredBkgStatErr': 14.4,
 'PredBkgSystErr': 13.8,
 'ExpSig': 366,
 'ExpSigStatErr': 2.9,
 'ExpSigSystErr': 0,
 'ExpBkg': 117,
 'BkgEstABMC': 0.71,
 'BkgEstABMCErr': 3.1,
 'BkgEstABData': 0.71,
 'BkgEstABDataErr': 11.0,
 'PredBkgMC': 129.7,
 'BkgEstMethodSystErr': 11.3,
 'BkgEstBkgCompSystErr': 6.1,
 'BkgEstTotalSystErr': 12.8,
 'BkgEstStatErr': 13.4,
 'BkgEstWJetsUpABMC': 0.67,
 'BkgEstWJetsUpABMCErr': 2.9,
 'BkgEstWJetsDownABMC': 0.74,
 'BkgEstWJetsDownABMCErr': 3.2,
 'BkgEstWJetsCompSyst': 5.5,
 'BkgEstBosonsUpABMC': 0.69,
 'BkgEstBosonsUpABMCErr': 4.5,
 'BkgEstBosonsDownABMC': 0.72,
 'BkgEstBosonsDownABMCErr': 2.6,
 'BkgEstBosonsCompSyst': 2.6,
 'PNetSidebandPredData': 137.4,
 'PNetSidebandPredDataStatErr': 21.8,
 'PNetSidebandData': 118,
 'PNetSidebandDataStatErr': 10.9,
 'SRTwoPredBkg': 5,
 'SRTwoPredBkgStatErr': 0.7,
 'SRTwoPredBkgSystErr': 1.7,
 'SRTwoBkgEstStatErr': 13.4,
 'SRTwoBkgEstSystErr': 35.7,
 'SRTwoExpSig': 99,
 'SRTwoExpSigStatErr': 1.5,
 'SRTwoExpSig